In [9]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from datetime import datetime

df = pd.read_csv("earthquke.csv")  

df["time"] = pd.to_datetime(df["time"])
df["timestamp"] = df["time"].apply(lambda x: x.timestamp())

label_encoders = {}
categorical_cols = ["magType", "locationSource", "magSource"]

for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

features = ["timestamp", "latitude", "longitude", "depth", "magType",
            "gap", "dmin", "rms", "horizontalError", "depthError", "magError", "locationSource", "magSource"]

target = "mag"  

feature_scaler = MinMaxScaler()
df[features] = feature_scaler.fit_transform(df[features])

target_scaler = MinMaxScaler()
df[target] = target_scaler.fit_transform(df[[target]])

def create_sequences(data, target_data, seq_length=20):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(target_data[i+seq_length])  
    return np.array(X), np.array(y)

seq_length = 20  
data_values = df[features].values
target_values = df[target].values

X, y = create_sequences(data_values, target_values, seq_length)

split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

model = Sequential([
    Bidirectional(LSTM(256, return_sequences=True), input_shape=(seq_length, len(features))),
    Dropout(0.2),
    LSTM(128, return_sequences=True),
    Dropout(0.2),
    LSTM(64, return_sequences=False),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=25, batch_size=32)

model.save("/mnt/data/earthquake_lstm_model.h5")

predictions = model.predict(X_test)
predicted_magnitude = target_scaler.inverse_transform(predictions)

print(predicted_magnitude[:10])


Epoch 1/25


C:\Users\DEVANSH\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


39/39 ━━━━━━━━━━━━━━━━━━━━ 7s 63ms/step - loss: 0.0860 - mae: 0.2371 - val_loss: 0.0421 - val_mae: 0.1834
Epoch 2/25
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step - loss: 0.0357 - mae: 0.1639 - val_loss: 0.0231 - val_mae: 0.1330
Epoch 3/25
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 50ms/step - loss: 0.0294 - mae: 0.1505 - val_loss: 0.0228 - val_mae: 0.1311
Epoch 4/25
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 50ms/step - loss: 0.0297 - mae: 0.1513 - val_loss: 0.0232 - val_mae: 0.1333
Epoch 5/25
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step - loss: 0.0319 - mae: 0.1540 - val_loss: 0.0230 - val_mae: 0.1322
Epoch 6/25
39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step - loss: 0.0306 - mae: 0.1545 - val_loss: 0.0228 - val_mae: 0.1309
Epoch 7/25
39/39 ━━━━━━━━━━━━━━━━━━━━ 3s 80ms/step - loss: 0.0310 - mae: 0.1549 - val_loss: 0.0230 - val_mae: 0.1321
Epoch 8/25
39/39 ━━━━━━━━━━━━━━━━━━━━ 4s 93ms/step - loss: 0.0310 - mae: 0.1536 - val_loss: 0.0229 - val_mae: 0.1316
Epoch 9/25
39/39 ━━━━━━━━━━━━━━━━━━━━ 3s 73ms/step - loss: 0.0314 - mae: 0.

10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step
[[3.8199797]
 [3.8199797]
 [3.8199797]
 [3.8199797]
 [3.8199797]
 [3.8199797]
 [3.8199797]
 [3.8199797]
 [3.8199797]
 [3.8199797]]


In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

df = pd.read_csv("earthquke.csv")  

df["time"] = pd.to_datetime(df["time"])
df["timestamp"] = df["time"].apply(lambda x: x.timestamp())

label_encoders = {}
categorical_cols = ["magType", "locationSource", "magSource"]

for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

features = ["timestamp", "latitude", "longitude", "depth", "mag", "magType",
            "gap", "dmin", "rms", "horizontalError", "depthError", "magError", "locationSource", "magSource"]

scaler = MinMaxScaler()
df[features] = scaler.fit_transform(df[features])

seq_length = 10

def create_sequences(data, seq_length=10):
    X = []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
    return np.array(X)

data_values = df[features].values
X_input = create_sequences(data_values, seq_length)

model = tf.keras.models.load_model("earthquake_lstm_model.h5", compile=False)

model.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredError(), metrics=['mae'])

predictions = model.predict(X_input)

predicted_magnitude = scaler.inverse_transform(
    np.concatenate((data_values[seq_length:, :4], predictions, data_values[seq_length:, 5:]), axis=1)
)[:, 4]

print(predicted_magnitude)


48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
[3.85117736 3.85117736 3.85117736 ... 3.85117736 3.85117736 3.85117736]


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# ✅ 1. Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_directory(
    'path_to_train_dataset',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    'path_to_test_dataset',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# ✅ 2. Load Pretrained EfficientNetB3 (instead of MobileNetV2)
base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
base_model.trainable = False  # Freeze base model

x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
x = Dropout(0.3)(x)  # Regularization
output = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

# ✅ 3. Compute Class Weights (to handle imbalance)
y_true = train_generator.classes
class_weights = compute_class_weight('balanced', classes=np.unique(y_true), y=y_true)
class_weight_dict = dict(enumerate(class_weights))

# ✅ 4. Compile Model with Adam Optimizer & Learning Rate Scheduler
optimizer = Adam(learning_rate=1e-3)

model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6, verbose=1
)

# ✅ 5. Train the Model
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=20,
    class_weight=class_weight_dict,  # Apply class weights
    callbacks=[lr_scheduler]
)

# ✅ 6. Evaluate Model
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")
